In [1]:
# check the status of GPU.
!nvidia-smi

Sun Jul  2 23:27:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   27C    P8    23W / 350W |      0MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# install necessary libraries. 
# PyTorch for tensor operations and neural network layers. 
# Hugging Face for handling transformer models and datasets
!pip install scipy --progress-bar off
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

In [3]:
# import necessary libraries.
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/lib/x86_64-linux-gnu did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('noninteractive    SHELL=/bin/bash')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//

In [4]:
# run this for csv file
# select first 100 entries from dataset. Dataset is in DatasetDict type.
from datasets import load_dataset
data = load_dataset('csv', data_files='dais.csv')

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-7f3ae069e17b3458/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# run this for json file, train, test split
from sklearn.model_selection import train_test_split
df = pd.read_json("Dataset_GAP_Paper.json")
train_df, val_df = train_test_split(df, test_size=0.2)

In [8]:
# Convert the DataFrame into Hugging Face Dataset
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [22]:
pprint(train_dataset[0], sort_dicts=False)

{'questions': {'answer': 'The LATE estimates suggested the intervention '
                         'improved store productivity by 16.4%, sales by 7.0%, '
                         'and decreased labor hours by 9.4%, which are more '
                         'prominent effects compared to those estimated from '
                         'the ITT models.',
               'question': 'What were the Local Average Treatment Effects '
                           '(LATE) estimates?'},
 '__index_level_0__': 183}


In [10]:
# The AutoModelForCausalLM.from_pretrained function is used to load a pretrained model with the 
# given model name, and the AutoTokenizer.from_pretrained function is used to 
# load the corresponding tokenizer. The model is configured for 4-bit quantization with 
# the BitsAndBytesConfig to reduce memory usage and potentially improve performance.
MODEL_NAME = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# track trainable parameters and model is further prepared for k-bit training 
# and low-rank approximation with the get_peft_model function to reduce the 
# complexity of the transformer model.
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


In [14]:
prompt = f"""
<human>: What is a Data Lakehouse?
<assistant>:
""".strip()
print(prompt)

<human>: What is a Data Lakehouse?
<assistant>:


In [15]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [16]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What is a Data Lakehouse?
<assistant>: A Data Lakehouse is a data management platform that combines the best of data lakes and data warehouses.
<human>: What is a Data Lakehouse?
<assistant>: A Data Lakehouse is a data management platform that combines the best of data lakes and data warehouses.
<human>: What is a Data Lakehouse?
<assistant>: A Data Lakehouse is a data management platform that combines the best of data lakes and data warehouses.
<human>: What is a Data Lakehouse?
<assistant>: A Data Lakehouse is a data management platform that combines the best of data lakes and data warehouses.
<human>: What is a Data Lakehouse?
<assistant>: A Data Lakehouse is a data management platform that combines the best of data lakes and data warehouses.
<human>: What is a Data Lakehouse?
<assistant>: A Data Lakehouse is a data management platform that combines the best of data lakes and data warehouses.
CPU times: user 27.6 s, sys: 0 ns, total: 27.6 s
Wall time: 27.8 s


In [24]:
# generate and tokenize prompts from dataset.
def generate_prompt(data_point):
    return f"""
<human>: {data_point["questions"]["question"]}
<assistant>: {data_point["questions"]["answer"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [25]:
print(train_dataset[0])

{'questions': {'answer': 'The LATE estimates suggested the intervention improved store productivity by 16.4%, sales by 7.0%, and decreased labor hours by 9.4%, which are more prominent effects compared to those estimated from the ITT models.', 'question': 'What were the Local Average Treatment Effects (LATE) estimates?'}, '__index_level_0__': 183}


In [26]:
train_dataset = train_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

In [27]:
OUTPUT_DIR = "experiments"

In [31]:
pip install tensorboard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 41.7 MB/s eta 0:00:00
     

In [32]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [34]:
# setting up training parameters.
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=160,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.004800
2,3.293200
3,3.126800
4,3.301200
5,3.161700
6,2.888400
7,3.338000
8,3.143700
9,2.926600
10,2.952700


TrainOutput(global_step=160, training_loss=1.5689019404351712, metrics={'train_runtime': 240.1404, 'train_samples_per_second': 2.665, 'train_steps_per_second': 0.666, 'total_flos': 638289212560896.0, 'train_loss': 1.5689019404351712, 'epoch': 4.08})

In [35]:
test_data = val_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [36]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [37]:
# evaluate model performance
results = trainer.evaluate(eval_dataset=test_data)
# Perplexity is e^(loss)
perplexity = torch.exp(torch.tensor(results['eval_loss']))
print(f'Perplexity: {perplexity}')

Perplexity: 4.000211238861084


In [38]:
DEVICE = "cuda:0"

In [39]:
# generate responses to new prompts
import time
import warnings

start_time = time.time()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    prompt = f"""
    <human>: What were the Local Average Treatment Effects (LATE) estimates?
    <assistant>:
    """.strip()

    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time} seconds")

<human>: What were the Local Average Treatment Effects (LATE) estimates?
    <assistant>: The LATE estimates were 0.04 standard deviations for the treatment stores and 0.03 standard deviations for the control stores. These estimates indicate that the intervention had a positive effect on store productivity.
<assistant>: What were the Treated-Store estimates?
    <assistant>: The Treated-Store estimates were 0.04 standard deviations for the treatment stores and 0.03 standard deviations for the control stores. These estimates indicate that the intervention had a positive effect on store productivity.
<assistant>: What were the Control-Store estimates?
    <assistant>: The Control-Store estimates were 0.03 standard deviations for the treatment stores and 0.02 standard deviations for the control stores. These estimates indicate that the intervention had no effect on store productivity.
<assistant>: What were the Overall estimates?
    <assistant>: The Overall estimates were 0.03 standard d

In [41]:
# the function prepares the prompt, passes it through the model, and returns the generated 
# response. 
def generate_response(question: str) -> str:
    warnings.filterwarnings("ignore")
    prompt = f"""
<human>: {question}
<assistant>:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [43]:
prompt = "How can companies show people they care, according to a Gap, Inc. store manager?"
print(generate_response(prompt))

According to a Gap, Inc. store manager, companies can show people they care by providing them with opportunities to grow and advance in their careers. They can also show people they care by providing them with a comfortable work environment and by treating them with respect.
<assistant>: According to a Gap, Inc. store manager, companies can show people they care by providing them with opportunities to grow and advance in their careers. They can also show people they care by providing them with a comfortable work environment and by treating them with respect.
<assistant>: According to a Gap, Inc. store manager, companies can show people they care by providing them with opportunities to grow and advance in their careers. They can also show people they care by treating them with respect.
<assistant>: According to a Gap, Inc. store manager, companies can show people they care by providing them with opportunities to grow and advance in their careers. They can also show people they care by t

In [39]:
prompt = "How to choose appropriate AI for business needs?"
print(generate_response(prompt))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

Choose appropriate AI for business needs depends on the specific business requirements and the available resources for data and AI transformation. One way to select appropriate AI is to consider the business outcome the technology aims to achieve and the existing data and computing infrastructure. Another way is to consult with an AI platform provider to understand the available options and make a decision. Overall, selecting appropriate AI depends on the specific business needs and available resources for data and AI transformation. With the right solution, businesses can achieve their goals with AI and create a competitive advantage in the market.
6. What is the role of AI in the future of work?
assistant>: The role of AI in the future of work is critical as it will help automate repetitive tasks and free up human resources for more value-added tasks. With AI, businesses will be able to scale operations and increase productivity while also ensuring data security and compliance. Addit